In [365]:
import pandas as pd
import numpy as np
from pathlib import Path

PATH = Path("books_after_merge.csv")  
df = pd.read_csv(PATH)
df.shape

/var/folders/1_/nzn6lhf551sdbvr_7pl_5b8c0000gn/T/ipykernel_995/3285503426.py:6: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH)


(25375, 66)

In [366]:
stats = pd.DataFrame({
    "column": df.columns,
    "non_null_cnt": [df[c].notna().sum() for c in df.columns],
    "null_cnt": [df[c].isna().sum() for c in df.columns],
    "null_ratio": [df[c].isna().mean() for c in df.columns],
    "nunique": [df[c].nunique(dropna=True) for c in df.columns],
    "dtype": [str(df[c].dtype) for c in df.columns],
})
stats["all_null"] = stats["null_cnt"] == len(df)
stats["almost_all_null"] = stats["null_ratio"] > 0.98
stats["is_constant"] = stats["nunique"] <= 1
stats["is_norm_like"] = stats["column"].str.contains(r'(?:^|_)norm(?:_|$)', case=False, regex=True)
stats["is_equal_flag"] = stats["column"].str.endswith("_equal_all")

def suffix_of(c):
    for suf in ("_litres","_litnet","_authoday"):
        if c.endswith(suf):
            return suf
    return ""
stats["suffix"] = stats["column"].apply(suffix_of)

stats.sort_values(["is_norm_like","is_equal_flag","all_null","almost_all_null","is_constant","null_ratio","column"],
                  ascending=[False,False,False,False,False,False,True]).head(20)

,column,non_null_cnt,null_cnt,null_ratio,nunique,dtype,all_null,almost_all_null,is_constant,is_norm_like,is_equal_flag,suffix
37,author_norm_y,2001,23374,0.921143,794,object,False,False,False,True,False,
38,title_norm_y,2001,23374,0.921143,1982,object,False,False,False,True,False,
54,title_norm,12038,13337,0.525596,8322,object,False,False,False,True,False,
53,author_norm,12040,13335,0.525517,1283,object,False,False,False,True,False,
25,author_norm_x,14062,11313,0.445833,3708,object,False,False,False,True,False,
26,title_norm_x,14063,11312,0.445793,11335,object,False,False,False,True,False,
65,age_equal_all,25375,0,0.000000,1,bool,False,False,True,False,True,
61,categories_equal_all,25375,0,0.000000,1,bool,False,False,True,False,True,
58,date_equal_all,25375,0,0.000000,1,bool,False,False,True,False,True,
63,description_equal_all,25375,0,0.000000,1,bool,False,False,True,False,True,


In [367]:

KEEP_CORE = [c for c in ["join_key","has_litres","has_litnet","has_authoday"] if c in df.columns]

KEEP_STRICT = [c for c in ["title","authors","price","date","rate","categories","description","age","views","likes","comments"]
               if c in df.columns]

KEEP_UNIQUE = [c for c in df.columns if c.endswith(("_litres","_litnet","_authoday"))
               and any(k in c for k in [
                   "isbn", "id_litnet", "url_litres",
                   "views_", "likes_", "comments_",
                   "rating_count", "reviews_count",
                   "release_date", "written_date",
                   "pages", "copyright_holder",
                   "formats", "cycle_", "exclusive_",
                   "symbols_count", "a4_sheets"
               ])]

DROP_TECH = stats.loc[stats["is_norm_like"] | stats["is_equal_flag"], "column"].tolist()
DROP_CONST = stats.loc[(stats["is_constant"]) & (~stats["column"].isin(KEEP_CORE)), "column"].tolist()
DROP_ALLNULL = stats.loc[stats["all_null"], "column"].tolist()

DROP = sorted(set(DROP_TECH + DROP_CONST + DROP_ALLNULL))

len(KEEP_CORE), len(KEEP_STRICT), len(KEEP_UNIQUE), len(DROP)

(4, 8, 18, 23)

In [368]:

cols_trim = [c for c in df.columns if c not in DROP]
df_trim = df[cols_trim].copy()

df_trim.shape, df_trim.columns


((25375, 43),
 Index(['join_key', 'has_litres', 'has_litnet', 'has_authoday', 'title',
        'authors', 'url_litres', 'title_litres', 'authors_litres',
        'rating_litres', 'rating_count_litres', 'reviews_count_litres',
        'price_litres', 'genres_litres', 'age_limit_litres',
        'release_date_litres', 'written_date_litres', 'pages_litres',
        'isbn_litres', 'copyright_holder_litres', 'formats_litres',
        'description_litres', 'id_litnet', 'title_litnet', 'author_litnet',
        'genre_litnet', 'price_litnet', 'year_litnet', 'description_litnet',
        'title_authoday', 'authors_authoday', 'categories_authoday',
        'date_authoday', 'symbols_count_authoday', 'a4_sheets_authoday',
        'views_authoday', 'likes_authoday', 'comments_authoday',
        'reviews_authoday', 'price_authoday', 'cycle_authoday',
        'exclusive_authoday', 'annotation_authoday'],
       dtype='object'))

In [369]:

split_cols = df_trim["join_key"].str.split(" \| ", n=1, expand=True)
split_cols.columns = ["author_from_key", "title_from_key"]

df_trim = pd.concat([df_trim, split_cols], axis=1)

if "authors" in df_trim.columns:
    df_trim["authors"] = df_trim["authors"].fillna(df_trim["author_from_key"])
else:
    df_trim["authors"] = df_trim["author_from_key"]

if "title" in df_trim.columns:
    df_trim["title"] = df_trim["title"].fillna(df_trim["title_from_key"])
else:
    df_trim["title"] = df_trim["title_from_key"]

df_trim = df_trim.drop(columns=["author_from_key", "title_from_key"])

df_trim[["join_key","authors","title"]].head(10)

<>:1: SyntaxWarning: invalid escape sequence '\|'
<>:1: SyntaxWarning: invalid escape sequence '\|'
/var/folders/1_/nzn6lhf551sdbvr_7pl_5b8c0000gn/T/ipykernel_995/2799399254.py:1: SyntaxWarning: invalid escape sequence '\|'
  split_cols = df_trim["join_key"].str.split(" \| ", n=1, expand=True)


,join_key,authors,title
0,4itaka | абсолютное знание развращает абсолютно,4itaka,абсолютное знание развращает абсолютно
1,4k zeus | rwby во имя силы юности я,4k zeus,rwby во имя силы юности я
2,ach | кросс,ach,кросс
3,afkter | эльфингард,afkter,эльфингард
4,alchoz | гастарбайтер,alchoz,гастарбайтер
5,alchoz | другой шинигами,alchoz,другой шинигами
6,aleks hom | система развития гильдии в мире fa...,aleks hom,система развития гильдии в мире fairy tail
7,aleksey nik | ворожея,aleksey nik,ворожея
8,aleksey nik | драконья кровь,aleksey nik,драконья кровь
9,alena milagro | омут,alena milagro,омут


In [370]:
df_trim = df_trim.drop(columns='join_key')

In [371]:
df_trim.to_csv("books_after_trim.csv", index=False)

In [372]:

UNIFY_FIELDS = {
    "title": ["title_litres", "title_authoday", "title_litnet"],
    "authors": ["authors_litres", "authors_authoday", "author_litnet"],
    "genre": ["genres_litres", "categories_authoday", "genre_litnet"],
    "description": ["description_litres", "annotation_authoday", "description_litnet"],
    "age": ["age_limit_litres", "age_litnet"],
}

def coalesce(df, cols):
    cols = [c for c in cols if c in df.columns]
    if not cols:
        return np.nan
    return df[cols].bfill(axis=1).iloc[:, 0]  

for new_col, src_cols in UNIFY_FIELDS.items():
    df_trim[new_col] = coalesce(df_trim, src_cols)
    filled = df_trim[new_col].notna().sum()
    print(f"{new_col}: объединено из {len(src_cols)} полей, непустых значений = {filled}")

to_drop = [c for cols in UNIFY_FIELDS.values() for c in cols if c in df_trim.columns]
df_trim = df_trim.drop(columns=to_drop)

print("\n🧹 Удалено старых колонок:", len(to_drop))
print(" Добавлены новые:", list(UNIFY_FIELDS.keys()))
df_trim.head(5)

title: объединено из 3 полей, непустых значений = 25375
authors: объединено из 3 полей, непустых значений = 25374
genre: объединено из 3 полей, непустых значений = 25375
description: объединено из 3 полей, непустых значений = 25375
age: объединено из 2 полей, непустых значений = 14063

🧹 Удалено старых колонок: 13
 Добавлены новые: ['title', 'authors', 'genre', 'description', 'age']


,has_litres,has_litnet,has_authoday,title,authors,url_litres,rating_litres,rating_count_litres,reviews_count_litres,price_litres,...,views_authoday,likes_authoday,comments_authoday,reviews_authoday,price_authoday,cycle_authoday,exclusive_authoday,genre,description,age
0,False,False,True,Абсолютное знание - развращает абсолютно,4itaka,NaN,NaN,NaN,NaN,NaN,...,36K,595,36,0.0,Бесплатно,Скайрим,Нет,"Роман, Фанфик, Фэнтези, Попаданцы",Продолжение похождений попаданца ступившего на...,NaN
1,False,False,True,"RWBY. Во Имя Силы Юности, Я...",4k_ZEUS,NaN,NaN,NaN,NaN,NaN,...,88K,1 314,388,0.0,Бесплатно,"Во Имя Силы Юности, Я...",Нет,"Роман, Фанфик, Фэнтези, Приключения",Попаданец в мир RWBY с внешностью Майто Гая из...,NaN
2,False,False,True,Кросс,Ach,NaN,NaN,NaN,NaN,NaN,...,410K,1 306,453,1.0,120 ₽,Ритм,Нет,"Роман, Космическая фантастика, Попаданцы в кос...","Присутствуют мотивы EVE. ГГ будет: пахать, пот...",NaN
3,False,False,True,Эльфингард,Afkter,NaN,NaN,NaN,NaN,NaN,...,393K,2 337,339,0.0,Бесплатно,Арда,Нет,"Роман, Фанфик, Попаданцы, Фэнтези",Это история о перерождении души человека в нов...,NaN
4,False,False,True,Гастарбайтер,alchoz,NaN,NaN,NaN,NaN,NaN,...,178K,910,125,0.0,Бесплатно,NaN,Нет,"Роман, Фанфик, Попаданцы, Попаданцы в магическ...","Очнуться в новом теле, в неизвестной лаборатор...",NaN


In [373]:
df_trim.columns

Index(['has_litres', 'has_litnet', 'has_authoday', 'title', 'authors',
       'url_litres', 'rating_litres', 'rating_count_litres',
       'reviews_count_litres', 'price_litres', 'release_date_litres',
       'written_date_litres', 'pages_litres', 'isbn_litres',
       'copyright_holder_litres', 'formats_litres', 'id_litnet',
       'price_litnet', 'year_litnet', 'date_authoday',
       'symbols_count_authoday', 'a4_sheets_authoday', 'views_authoday',
       'likes_authoday', 'comments_authoday', 'reviews_authoday',
       'price_authoday', 'cycle_authoday', 'exclusive_authoday', 'genre',
       'description', 'age'],
      dtype='object')

In [374]:
df_final = df_trim.copy()


DROP_PATTERNS = [
    "id_",
    "isbn",
    "url",
]

drop_extra = [c for c in df_final.columns if any(pat in c for pat in DROP_PATTERNS)]

print(f" Удаляем технические поля ({len(drop_extra)} шт.):")
for c in drop_extra:
    print(" -", c)

# удаляем
df_final = df_final.drop(columns=drop_extra)

print(f"\n Осталось {len(df_final.columns)} колонок после очистки")
df_final.head(5)

 Удаляем технические поля (3 шт.):
 - url_litres
 - isbn_litres
 - id_litnet

 Осталось 29 колонок после очистки


,has_litres,has_litnet,has_authoday,title,authors,rating_litres,rating_count_litres,reviews_count_litres,price_litres,release_date_litres,...,views_authoday,likes_authoday,comments_authoday,reviews_authoday,price_authoday,cycle_authoday,exclusive_authoday,genre,description,age
0,False,False,True,Абсолютное знание - развращает абсолютно,4itaka,NaN,NaN,NaN,NaN,NaN,...,36K,595,36,0.0,Бесплатно,Скайрим,Нет,"Роман, Фанфик, Фэнтези, Попаданцы",Продолжение похождений попаданца ступившего на...,NaN
1,False,False,True,"RWBY. Во Имя Силы Юности, Я...",4k_ZEUS,NaN,NaN,NaN,NaN,NaN,...,88K,1 314,388,0.0,Бесплатно,"Во Имя Силы Юности, Я...",Нет,"Роман, Фанфик, Фэнтези, Приключения",Попаданец в мир RWBY с внешностью Майто Гая из...,NaN
2,False,False,True,Кросс,Ach,NaN,NaN,NaN,NaN,NaN,...,410K,1 306,453,1.0,120 ₽,Ритм,Нет,"Роман, Космическая фантастика, Попаданцы в кос...","Присутствуют мотивы EVE. ГГ будет: пахать, пот...",NaN
3,False,False,True,Эльфингард,Afkter,NaN,NaN,NaN,NaN,NaN,...,393K,2 337,339,0.0,Бесплатно,Арда,Нет,"Роман, Фанфик, Попаданцы, Фэнтези",Это история о перерождении души человека в нов...,NaN
4,False,False,True,Гастарбайтер,alchoz,NaN,NaN,NaN,NaN,NaN,...,178K,910,125,0.0,Бесплатно,NaN,Нет,"Роман, Фанфик, Попаданцы, Попаданцы в магическ...","Очнуться в новом теле, в неизвестной лаборатор...",NaN


In [375]:
df_final.columns

Index(['has_litres', 'has_litnet', 'has_authoday', 'title', 'authors',
       'rating_litres', 'rating_count_litres', 'reviews_count_litres',
       'price_litres', 'release_date_litres', 'written_date_litres',
       'pages_litres', 'copyright_holder_litres', 'formats_litres',
       'price_litnet', 'year_litnet', 'date_authoday',
       'symbols_count_authoday', 'a4_sheets_authoday', 'views_authoday',
       'likes_authoday', 'comments_authoday', 'reviews_authoday',
       'price_authoday', 'cycle_authoday', 'exclusive_authoday', 'genre',
       'description', 'age'],
      dtype='object')

In [376]:
mask = df_final["price_litres"].notna() & df_final["price_authoday"].notna()
count_both = mask.sum()

count_both

np.int64(2603)

In [377]:
import re
import numpy as np

def normalize_price(x):
    """Очищает цену от символов валюты, слов 'бесплатно', и приводит к float."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    if any(word in s for word in ["бесплат", "free"]):
        return 0.0
    s = re.sub(r"[^0-9,\.]", "", s).replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

numeric_cols = ["price_litres", "price_authoday", "price_litnet"]
for c in numeric_cols:
    if c in df_final.columns:
        df_final[c] = df_final[c].apply(normalize_price)

In [378]:
df_final[["price_litres","price_authoday","price_litnet",
                "rating_litres","rating_count_litres","reviews_count_litres",
                "views_authoday","likes_authoday","comments_authoday",
                "reviews_authoday","a4_sheets_authoday","symbols_count_authoday"]].sample(10)

,price_litres,price_authoday,price_litnet,rating_litres,rating_count_litres,reviews_count_litres,views_authoday,likes_authoday,comments_authoday,reviews_authoday,a4_sheets_authoday,symbols_count_authoday
10192,NaN,150.0,NaN,NaN,NaN,NaN,58K,833,749,0.0,"10,97",438799.0
23728,199.0,NaN,NaN,5.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
19833,NaN,127.0,NaN,NaN,NaN,NaN,94K,654,116,0.0,"11,44",457685.0
18122,490.0,NaN,NaN,5.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
23623,299.0,NaN,NaN,4.6,279.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN
8374,NaN,169.0,NaN,NaN,NaN,NaN,179K,1 245,386,0.0,"12,08",483185.0
4177,164.0,NaN,NaN,5.0,10.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
16769,249.0,NaN,NaN,3.5,110.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN
5865,249.0,NaN,NaN,4.9,26.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
15076,NaN,159.0,NaN,NaN,NaN,NaN,170K,1 312,420,0.0,"12,02",480776.0


In [379]:
df_final = df_final.drop_duplicates()

In [380]:
df_final.to_csv("books_after_trim.csv", index=False)

In [381]:

flags = df_final[["has_litres", "has_litnet", "has_authoday"]].astype(bool)


print("Кол-во книг по источникам:")
print(flags.sum().to_frame("count"))

from itertools import combinations

pairs = list(combinations(["has_litres", "has_litnet", "has_authoday"], 2))

print("\n Пересечения (попарно):")
for a, b in pairs:
    both = (flags[a] & flags[b]).sum()
    only_a = (flags[a] & ~flags[b]).sum()
    only_b = (~flags[a] & flags[b]).sum()
    print(f"{a} ∩ {b}: {both:>5} (только {a}: {only_a:>5}, только {b}: {only_b:>5})")

all_three = (flags["has_litres"] & flags["has_litnet"] & flags["has_authoday"]).sum()
print(f"\n Во всех трёх источниках: {all_three}")

import pandas as pd

summary = pd.DataFrame({
    "Source": ["LitRes","LitNet","Author.Today","LitRes ∩ LitNet","LitRes ∩ Author.Today","LitNet ∩ Author.Today","All three"],
    "Count": [
        flags["has_litres"].sum(),
        flags["has_litnet"].sum(),
        flags["has_authoday"].sum(),
        (flags["has_litres"] & flags["has_litnet"]).sum(),
        (flags["has_litres"] & flags["has_authoday"]).sum(),
        (flags["has_litnet"] & flags["has_authoday"]).sum(),
        all_three
    ]
})

summary

Кол-во книг по источникам:
              count
has_litres    14063
has_litnet     2001
has_authoday  12039

 Пересечения (попарно):
has_litres ∩ has_litnet:   122 (только has_litres: 13941, только has_litnet:  1879)
has_litres ∩ has_authoday:  2603 (только has_litres: 11460, только has_authoday:  9436)
has_litnet ∩ has_authoday:     8 (только has_litnet:  1993, только has_authoday: 12031)

 Во всех трёх источниках: 4


,Source,Count
0,LitRes,14063
1,LitNet,2001
2,Author.Today,12039
3,LitRes ∩ LitNet,122
4,LitRes ∩ Author.Today,2603
5,LitNet ∩ Author.Today,8
6,All three,4
